# Calculating Semantic Relatedness using Wikipedia

* **Armin Sajadi** (sajadi@cs.dal.ca)

This is a simple and step by step explanation of calculating semantic relatedness using Wikipedia. We start by preprocessing and building the api, that is explained in the following papers papers:





# Read Here First
** Make sure you have already followed [the first 2 Steps](../README.md#hosting) of the setup and have all the requirements before trying to setup to run these scripts **


# Table of Context

**[Wikipedia Interface](#Wikipedia-Interface)**

**[Fast Pagerank Implementation](#Fast-Pagerank-Implementation)**

**[Calculating Semantic Relatedness](#Calculating-Semantic-Relatedness)**

**[A Simple Example](#A-Simple-Example)**

**[Calculating All The Embeddngs](#Calculating-All-The-Embeddngs)**

**[Visualizing The Embeddings](#Visualizing-The-Embeddings)**



# Wikipedia Interface
This is the main interface to Wikipedia database and provides basic functions given a pages, such as its:

* id or title
* synonym ring
* linkage
* in or out neighborhood. 

**You might need to modify, user, password and portnumbers**


In [ ]:
%%writefile wikipedia.py 
"""A General Class to interact with Wiki datasets"""
# uncomment

import sys;
import os
import scipy as sp
import pandas as pd
import cPickle as pickle
import MySQLdb
from collections import defaultdict

from utils import * # uncomment

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"


DISABLE_CACHE=False;
MAX_GRAPH_SIZE=1000000

DIR_IN=0;
DIR_OUT=1;
DIR_BOTH=2;
_db = MySQLdb.connect(host="127.0.0.1",port=3307,user='root',passwd="emilios",db="enwiki20160305")
_cursor = _db.cursor()
#WIKI_SIZE = 10216236;
#WIKI_SIZE = 13670498; #2016
WIKI_SIZE = 5576365; #no redirect, 2016
def close():
    global _db, _cursor;
    if _cursor is not None: 
        _cursor.close();
        _db.close();
    _cursor=_db=None;
def reopen():
    global _db, _cursor;
    if _db is None:
        _db = MySQLdb.connect(host="127.0.0.1",port=3307,user='root',passwd="emilios",db="enwiki20160305")
        _cursor = _db.cursor()

def disable_cache():
    global DISABLE_CACHE
    DISABLE_CACHE=True
def enable_cache():
    global DISABLE_CACHE
    DISABLE_CACHE=False
        
def load_table(tbname, limit=-1):
    """ Returns a list, containing a whole table     
    
    Args: 
        tbname: Table Name
    Returns: 
        The list of rows
    """
    if limit!=-1:
        q = """SELECT * FROM `%s` limit %s""" % (tbname, limit)
    else:
        q = """SELECT * FROM `%s`""" % (tbname,)
        
    _cursor.execute(q)
    rows = _cursor.fetchall();
    return rows
    
    
def id2title(wid):
    """ Returns the title for a given id

    Args: 
        wid: Wikipedia id       
    Returns: 
        The title of the page
    """
    title=None;

    _cursor.execute("""SELECT * FROM `page` where page_id = %s""", (wid,))
    row= _cursor.fetchone();
    if row is not None:
        title=row[2];          
    return title;

def ids2title(wids):
    """ Returns the titles for given list of wikipedia ids 

    Args: 
        wids: A list of Wikipedia ids          
    Returns: 
        The list of titles
    """
    if not wids:
        return []
    wid_list = [str(wid) for wid in wids] ;
    order = ','.join(['page_id'] + wid_list) ;
    wid_str = ",".join(wid_list)
    query = "SELECT page_id, page_title FROM `page` where page_id in ({0})" \
    .format(wid_str, order);
    _cursor.execute(query);
    rows = _cursor.fetchall();
    rows_dict = defaultdict(lambda: None, rows)
    titles = [rows_dict[wid] for wid in wids]
    return titles;

def encode_for_db(instr):
    if isinstance(instr, unicode):
        instr = instr.encode('utf-8')  
    return instr
        
def normalize_str(title):
    
    title = encode_for_db(title)
    title = title.replace(' ','_')
    return title
def title2id(title):
    """ Returns the id for a given title

    Args: 
        wid: Wikipedia id          
    Returns: 
        The title of the page
    """        
    if title is None:
        return None
    wid=None;
    title = normalize_str(title)
    _cursor.execute("""SELECT * FROM `page` where page_title=%s and page_namespace=0""", (title,))
    row= _cursor.fetchone();
    if row is not None:
        wid = getredir_id(row[0]) if row[3] else row[0];
    return wid;

def is_ambiguous(wid):
    _cursor.execute("""SELECT * FROM `categorylinks` WHERE `categorylinks`.cl_from=%s and `categorylinks`.cl_to=19204864;""", (wid,))
    row= _cursor.fetchone();
    return not (row is None)    

def getredir_id(wid):
    """ Returns the target of a redirected page 

    Args:
        wid: wikipedia id of the page
    Returns:
        The id of the target page
    """
    rid=None

    _cursor.execute("""select * from redirect where rd_from=%s;""", (wid,));
    row= _cursor.fetchone();
    if row is not None:
        rid=row[1]
    return rid 

def resolveredir(wid):
    tid = getredir_id(wid);
    if tid is not None:
        wid = tid;    
    return wid

def getredir_title(wid):
    """ Returns the target title of a redirected page 

    Args:
        wid: wikipedia id of the page
    Returns:
        The title of the target page
    """
    
    title=None;
    _cursor.execute(""" select page_title from redirect INNER JOIN page
                  on redirect.rd_to = page.page_id 
                  where redirect.rd_from =%s;""", (wid));
    row=_cursor.fetchone()
    if row is not  None:
        title=row[0];
    return title;

def synonymring_titles(wid):
    """ Returns the synonim ring of a page

    Example: synonymring_titles('USA')={('U.S.A', 'US', 'United_States_of_America', ...)}

    Args:
        wid: the wikipedia id
    Returns:
        all the titles in its synonym ring
    """
    wid = resolveredir(wid)
    _cursor.execute("""(select page_title from page where page_id=%s) union 
                 (select page_title from redirect INNER JOIN page
                    on redirect.rd_from = page.page_id 
                    where redirect.rd_to =%s);""", (wid,wid));
    rows=_cursor.fetchall();
    if rows:
        rows = tuple(r[0] for r in rows)
    return rows;


def anchor2concept(anchor):
    """ Returns the targets of an anchor text

    Args:
        anchor: anchor
        
    Returns:
        The list of the titles of the linked pages
    """
  
    anchor = encode_for_db(anchor)
        
    _cursor.execute("""select anchors.id, anchors.freq from anchors inner join page on anchors.id=page.page_id where anchors.anchor=%s;""", (anchor,))
    rows =_cursor.fetchall()
#     if rows:
#         rows = tuple(r[0] for r in rows)
    return rows


def id2anchor(wid):
    """ Returns the targets of an anchor text

    Args:
        anchor: anchor
        
    Returns:
        The list of the titles of the linked pages
    """
    _cursor.execute("""select anchor , freq from anchors where id=%s""", (wid,))
    rows =_cursor.fetchall()
#     if rows:
#         rows = tuple(r[0] for r in rows)
    return rows


def _getlinkedpages_query(id, direction):
    query="(SELECT {0} as lid FROM pagelinks where ({1} = {2}))"
    if direction == DIR_IN:
        query=query.format("pl_from","pl_to",id);
    elif direction == DIR_OUT:
        query=query.format("pl_to","pl_from",id);
    return query;

def getlinkedpages(wid,direction):
    """ Returns the linkage for a node

    Args:
        id: the wikipedia id
        direction: 0 for in, 1 for out, 2 for all
    Returns:
        The list of the ids of the linked pages
    """
    _cursor.execute(_getlinkedpages_query(wid, direction));
    rows =_cursor.fetchall()
    if rows:
        rows = tuple(r[0] for r in rows)
    return rows

def e2i(wids):
    elist=[];
    edict=dict();
    last=0;    
    for wid in itertools.chain(*iters):
        if wid not in edict:
            edict[wid]=last;
            elist.append(wid);
            last +=1; 
    return elist, edict;

def getneighbors(wid, direction):
    """ Returns the neighborhood for a node

    Args:
        id: the wikipedia id
        direction: 0 for in, 1 for out, 2 for all
    Returns:
        The vector of ids, and the 2d array sparse representation of the graph, in the form of
        array([[row1,col1],[row2, col2]]). This form is flexible for general use or be converted to scipy.sparse 
        formats
    """
    log('[getneighbors started]\twid = %s, direction = %s', wid, direction)
    
    idsquery = """(select  {0} as lid) union {1}""".format(wid,_getlinkedpages_query(wid,direction));

    _cursor.execute(idsquery);


    rows = _cursor.fetchall();
    if len(rows)<2:
        log('[getneighbors]\tERROR: empty')
        return (), sp.array([])
    
    
    neighids = tuple(r[0] for r in rows);
    if len(neighids)>MAX_GRAPH_SIZE:
        log('[getneighbors]\tERROR: too big, %s neighbors', len(neighids))
        return (), sp.array([])

    
    id2row = dict(zip(neighids, range(len(neighids))))

    neighbquery=  """select lid,pl_to as n_l_to from
                     ({0}) a  inner join
                     pagelinks on lid=pl_from""".format(idsquery);

    links=_cursor.execute(neighbquery);

    links = _cursor.fetchall();
    
    #links = tuple((id2row(u), id2row(v)) for u, v in links if (u in id2row) and (v in id2row));
    links = sp.array([[id2row[u], id2row[v]] for u, v in links if (u in id2row) and (v in id2row)]);
    
    log('Graph extracted, %s nodes and %s linkes', len(neighids), len(links) )
    log('[getneighbors]\tfinished')
    return (neighids,links)

def deletefromcache(wid, direction):
    wid = resolveredir(wid)
    if direction in [DIR_IN, DIR_BOTH] : 
        query =    """delete from {0} where cache_id={1}""".format('pagelinksorderedin', wid) 
        _cursor.execute(query);
    if direction in [DIR_OUT, DIR_BOTH]: 
        query =    """delete from {0} where cache_id={1}""".format('pagelinksorderedout', wid) 
        _cursor.execute(query);
    
def clearcache():
    if DISABLE_CACHE:
        return;
    _cursor.execute("delete  from pagelinksorderedin");
    _cursor.execute("delete  from pagelinksorderedout");

def checkcache(wid, direction):
    log('[checkcache started]\twid = %s, direction = %s', wid, direction)
    if DISABLE_CACHE:
        log('[checkcache]\tDisabled')
        return None
    

    
    em=None
    
    if direction == DIR_IN: 
        tablename = 'pagelinksorderedin';
        colname = 'in_neighb'
    elif direction == DIR_OUT: 
        tablename = 'pagelinksorderedout';
        colname = 'out_neighb';
    query =    """select {0} from {1} where cache_id={2}""".format(colname, tablename, wid)
    _cursor.execute(query);
    row = _cursor.fetchone();
    if row is not None:
        values, index = pickle.loads(row[0])
        log('[checkcache]\tfound')
        if not index:        
            log('[checkcache]\tempty embedding')
        em=pd.Series(values, index=index)
    else:
        log('[checkcache]\tnot found')

    log('[checkcache]\tfinished')
    return em


def cachescores(wid, em, direction):
    log('[cachescores started]\twid = %s, direction = %s', wid, direction)
    if DISABLE_CACHE:
        log('[cachescores]\tDisabled')
        return

    if direction == DIR_IN: 
        tablename = 'pagelinksorderedin';
        colname = 'in_neighb'

    elif direction == DIR_OUT: 
        tablename = 'pagelinksorderedout';
        colname = 'out_neighb';
        
    idscstr = pickle.dumps((em.values.tolist(), em.index.values.tolist()), pickle.HIGHEST_PROTOCOL)
    _cursor.execute("""insert into %s values (%s,'%s');""" %(tablename, wid, _db.escape_string(idscstr)));
    
    
    log('cachescores finished')


# Utils
Some small helper function for reporting purposes. 

In [ ]:
%%writefile utils.py 
"""Utility functions"""
# uncomment

import os
import re
import itertools
import scipy as sp
import pandas as pd
import datetime

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"

dirname = os.path.dirname(__file__)

def readds(url, usecols=None):    
    data = pd.read_table(url, header=None, usecols=usecols)
    return data

DISABLE_LOG=False;

def clearlog(logfile):
    with open(logfile, 'w'):
        pass;

def logres(outfile, instr, *params):
    outstr = instr % params;
    with open(outfile, 'a') as f:
        f.write("[%s]\t%s\n" % (str(datetime.datetime.now()) , outstr));          
        
def log(instr, *params):
    if DISABLE_LOG:
        return
    logres(logfile, instr, *params)
    
outdir = os.path.join(dirname, '../out')    
if not DISABLE_LOG:    
    logfile=os.path.join(outdir, 'log.txt');
    if not os.path.exists(logfile):
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        log('log created') 
        os.chmod(logfile, 0777)    
    
    
def timeformat(sec):
    return datetime.timedelta(seconds=sec)

def str2delta(dstr):
    r=re.match(('((?P<d>\d+) day(s?), )?(?P<h>\d+):(?P<m>\d+):(?P<s>\d*\.\d+|\d+)'),dstr)
    d,h,m,s=r.group('d'),r.group('h'),r.group('m'),r.group('s')
    d=int(d) if d is not None else 0
    h,m,s = int(h), int(m), float(s)    
    return datetime.timedelta(days=d, hours=h, minutes=m, seconds=s)

# Fast Pagerank Implementation

Here we have the actuall implementation of pagerank. Two implemenation are provided, both inspired  by the sparse fast solutions given in **Cleve Moler**'s book, [*Experiments with MATLAB*](http://www.mathworks.com/moler/index_ncm.html). The power method is much faster with enough precision for our task. Our benchmarsk shows that this implementation is faster than networkx implementation magnititude of times

The input is a 2d array, each row of the array is an edge of the graph [[a,b], [c,d]], a and b are the node numbers. 
(In case you want to caclulate reall page rank, uncomment the line that transposes the adjacency matrix)

In [ ]:
%%writefile pagerank.py 

"""Two "fast" implementations of PageRank.

Pythom implementations of Matlab original in Cleve Moler, Experiments with MATLAB.
"""
# uncomment
from __future__ import division

import scipy as sp
import scipy.sparse as sprs
import scipy.spatial
import scipy.sparse.linalg 


__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"


def create_csr(Z):
    """ Creates a csr presentation from 2darray presentation
    Args:
        Z: input graph in the form of a 2d array, such as sp.array([[2,0], [1,2], [2,1]])
    Returns:
        a csr representation
    
    """   
    rows = Z[:,0];
    cols = Z[:,1];
    n = max(max(rows), max(cols))+1;
    G=sprs.csr_matrix((sp.ones(rows.shape),(rows,cols)), shape=(n,n));
    return G

def moler_pagerank_sparse(G, p=0.85, personalize=None, reverse=False):
    """ Calculates pagerank given a csr graph
    
    Args:
        G: a csr graph.
        p: damping factor
        personlize: if not None, should be an array with the size of the nodes
                    containing probability distributions. It will be normalized automatically
        reverse: If true, returns the reversed-pagerank 
        
    Returns:
        Pagerank Scores for the nodes
     
    """
    # In Moler's algorithm, $G_{ij}$ represents the existences of an edge
    # from node $j$ to $i$, while we have assumed the opposite!
    if not reverse: 
        G=G.T;

    n,n=G.shape
    c=sp.asarray(G.sum(axis=0)).reshape(-1)
    r=sp.asarray(G.sum(axis=1)).reshape(-1)

    k=c.nonzero()[0]

    D=sprs.csr_matrix((1/c[k],(k,k)),shape=(n,n))

    if personalize is None:
        personalize=sp.ones(n) 
    personalize=personalize.reshape(n,1)
    e=(personalize/personalize.sum())*n 

    I=sprs.eye(n)
    x = sprs.linalg.spsolve((I - p*G.dot(D)), e);

    x=x/x.sum()
    return x
def moler_pagerank_sparse_power(G, p=0.85, max_iter = 100,  tol=1e-03,personalize=None, reverse=False):
    """ Calculates pagerank given a csr graph
    
    Args:
        G: a csr graph.
        p: damping factor
        max_iter: maximum number of iterations
        personlize: if not None, should be an array with the size of the nodes
                    containing probability distributions. It will be normalized automatically
        reverse: If true, returns the reversed-pagerank 
        
    Returns:
        Pagerank Scores for the nodes
     
    """
    # In Moler's algorithm, $G_{ij}$ represents the existences of an edge
    # from node $j$ to $i$, while we have assumed the opposite!
    if not reverse: 
        G=G.T;

    n,n=G.shape
    c=sp.asarray(G.sum(axis=0)).reshape(-1)
    r=sp.asarray(G.sum(axis=1)).reshape(-1)

    k=c.nonzero()[0]

    D=sprs.csr_matrix((1/c[k],(k,k)),shape=(n,n))

    if personalize is None:
        personalize=sp.ones(n) 
    personalize=personalize.reshape(n,1)
    e=(personalize/personalize.sum())*n 
    
    
    z = (((1-p)*(c!=0) + (c==0))/n)[sp.newaxis,:]
    G = p*G.dot(D)
    
    x = e/n 
    oldx = sp.zeros((n,1));
    
    iteration = 0
    
    while sp.linalg.norm(x-oldx) > tol:
        oldx = x
        x = G.dot(x) + e.dot(z.dot(x))
        iteration += 1
        if iteration >= max_iter:
            break;
    x = x/sum(x)
    
    return x.reshape(-1)

In [ ]:
%%writefile embedding.py

from __future__ import division


from wikipedia import * # uncomment
from pagerank import * # uncomment
import gensim

#from utils import * # uncomment

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"

_word2vec_model = None
def getword2vec_model():
    """ returns the word2vec model
    """
    
    return _word2vec_model

def conceptrep(wid, method ='rvspagerank', direction=DIR_BOTH, get_titles=True, cutoff=None):
    """ Calculates well-known similarity metrics between two concepts 
    Arg:
        id1, id2: the two concepts 
        method:
            rvspagerank: rvs-pagerank embedding
            word2vec : wor2vec representation
    Returns:
        The similarity score        
    """
        
    if method =='rvspagerank':
        return conceptrep_rvs(wid, direction, get_titles, cutoff)
    if 'word2vec' in method:
        return getentity2vector(wid)


def concept_embedding(wid, direction):
    """ Calculates concept embedding to be used in relatedness
    
    Args:
        wid: wikipedia id
        direction: 0 for in, 1 for out, 2 for all
        
    Returns:
        The neighbor ids, their scores and the whole neighorhood graph (for visualization purposes)
        
    """
    log('[concept_embedding started]\twid = %s, direction = %s', wid, direction)

    if direction == DIR_IN or direction==DIR_OUT:
        em = _concept_embedding_io(wid, direction)
    if direction == DIR_BOTH:
        em = _concept_embedding_both(wid, direction)
    log('[concept_embedding]\tfinished')
    return em
    
def _concept_embedding_io(wid, direction):
    wid = resolveredir(wid)
    cached_em = checkcache(wid, direction);
    if cached_em is not None:
        return cached_em;

    (ids, links) = getneighbors(wid, direction);
    if ids:
        scores = moler_pagerank_sparse_power(create_csr(links), reverse=True)
        em = pd.Series(scores, index=ids) 
    else:
        em = pd.Series([], index=[])  
    cachescores(wid, em, direction);
    return em
            

def _concept_embedding_both(wid, direction):            
        in_em = _concept_embedding_io(wid, DIR_IN);
        out_em = _concept_embedding_io(wid, DIR_OUT )
        if (in_em is None) or (out_em is None):
            return None;
        return in_em.add(out_em, fill_value=0)/2

def conceptrep_rvs(wid, direction, get_titles=True, cutoff=None):
    """ Finds a representation for a concept
    
        Concept Representation is a vector of concepts with their score
    Arg:
        wid: Wikipedia id
        direction: 0 for in, 1 for out, 2 for all
        titles: include titles in the embedding (not needed for mere calculations)
        cutoff: the first top cutoff dimensions (None for all)
        
    Returns:
        the vecotr of ids, their titles and theirs scores. It also returns the
        graph for visualization purposes. 
    """
    
    log('[conceptrep started]\twid = %s, direction = %s', wid, direction)
    
    em=concept_embedding(wid, direction);    
    if em.empty:
        return em;
    
    
    #ids = em.keys();
    
    if cutoff is not None:
        em = em.sort_values(ascending=False)
        em = em[:cutoff]
    if get_titles:
        em = pd.Series(zip(ids2title(em.index.tolist()), em.values.tolist()), index=em.index)
    log ('[conceptrep]\tfinished')
    return em

def gensim_loadmodel(model_path):
    """ Loads the word2vec model 
    Arg:
        model_path: path to the model
    """
    global _word2vec_model
    log('[getsim_word2vec]\tloading: %s', model_path)
    _word2vec_model = gensim.models.Word2Vec.load(model_path)                
    log('[getsim_word2vec]\loaded')
    return _word2vec_model
    
def getentity2vector(wid):
    if (wid is None) or (wid not in _word2vec_model.vocab):
        return  pd.Series(sp.zeros(_word2vec_model.vector_size))
    return pd.Series(_word2vec_model[wid])

def getword2vector(word):
    if word not in _word2vec_model.vocab:
        return  pd.Series(sp.zeros(_word2vec_model.vector_size))
    return pd.Series(_word2vec_model[word])
    


## Calculating Semantic Relatedness
The idea is get the neighborhood graph for each concept and calculating the similarity by embedding the graph into a vector and then perforiming cosine similarity. 

The process can be illustrated like this:
    ![](../resrc/alg.jpg)

In [ ]:
%%writefile calcsim.py 
"""Calculating Relatedness."""
# uncomment

from __future__ import division

from embedding import *
#from collections import defaultdict
import json
import math
from scipy import stats
import requests
from config import *

#from utils import * # uncomment

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"

#constants


def _unify_ids_scores(*id_sc_tuple):
    uids, id2in = e2i(*(ids for ids, _ in id_sc_tuple));
    
    uscs=tuple();            
    for ids,scs in id_sc_tuple:
        scs_u=sp.zeros(len(id2in))
        scs_u[[id2in[wid] for wid in ids]] = scs;            
        uscs += (scs_u,)                
    return uids, uscs       

def get_solr_count(s):
    """ Gets the number of documents the string occurs 
        NOTE: Multi words should be quoted
    Arg:
        s: the string (can contain AND, OR, ..)
    Returns:
        The number of documents
    """
    q='+text:(%s)'%(s,)
    qstr = 'http://localhost:8983/solr/enwiki20160305/select'
    params={'indent':'on', 'wt':'json', 'q':q, 'rows':0}
    r = requests.get(qstr, params=params)
    D = r.json()['response']
    return D['numFound']

def getsim_ngd(term1,term2):
    """ Calculates Normalized Google Distance (ngd) similarity between two concepts 
    Arg:
        id1, id2: the two concepts 
    Returns:
        The similarity score        
    """
    
    f1=get_solr_count('"%s"'%(term1,))
    
    f2=get_solr_count('"%s"'%(term2,))
    f12=get_solr_count('"%s" AND "%s"'%(term1, term2))
    
    if (f1==0) or (f2==0) or (f12==0):
        return 0;
    dist = (sp.log(max(f1,f2))-sp.log(f12))/(sp.log(WIKI_SIZE)-sp.log(min(f1,f2)));
    sim = 1-dist if dist <=1 else 0
    return sim
        
def getsim_word2vec(id1, id2):
    """ Calculates wor2vec similarity between two concepts 
    Arg:
        id1, id2: the two concepts 
    Returns:
        The similarity score        
    """
    model  = getword2vec_model()
    if model is None:
        log('[getsim_word2vec]\tmodel not loaded')
        raise Exception('model not loaded, try gensim_loadmodel()')
        
    if id1 not in model.vocab:
        #print '%s,%s skipped, %s not in vocab ' % (id1, id2, id1)
        return 0
    if id2 not in model.vocab:
        #print '%s,%s skipped, %s not in vocab ' % (id1, id2, id2)
        return 0
    return model.similarity(id1, id2)


def getsim_wlm(id1, id2):
    """ Calculates wlm (ngd) similarity between two concepts 
    Arg:
        id1, id2: the two concepts 
    Returns:
        The similarity score        
    """
    in1 = set(getlinkedpages(id1, DIR_IN))
    in2 = set(getlinkedpages(id2, DIR_IN))
    f1 = len(in1)
    f2 = len(in2)
    f12=len(in1.intersection(in2))
    if (f1==0) or (f2==0) or (f12==0):
        return 0;
    dist = (sp.log(max(f1,f2))-sp.log(f12))/(sp.log(WIKI_SIZE)-sp.log(min(f1,f2)));
    sim = 1-dist if dist <=1 else 0
    return sim

def getsim_cocit(id1, id2):
    """ Calculates co-citation similarity between two concepts 
    Arg:
        id1, id2: the two concepts 
    Returns:
        The similarity score        
    """
    in1 = set(getlinkedpages(id1, DIR_IN))
    in2 = set(getlinkedpages(id2, DIR_IN))
    f1 = len(in1)
    f2 = len(in2)
    if (f1==0) or (f2==0):
        return 0;
    
    f12=len(in1.intersection(in2))
    sim = (f12)/(f1+f2-f12);
    return sim


def getsim_coup(id1, id2):
    """ Calculates coupler similarity between two concepts 
    Arg:
        id1, id2: the two concepts 
    Returns:
        The similarity score        
    """
    in1 = set(getlinkedpages(id1, DIR_OUT))
    in2 = set(getlinkedpages(id2, DIR_OUT))
    f1 = len(in1)
    f2 = len(in2)
    if (f1==0) or (f2==0):
        return 0;
    
    f12=len(in1.intersection(in2))
    sim = (f12)/(f1+f2-f12);
    return sim

def getsim_ams(id1, id2):
    """ Calculates amlser similarity between two concepts 
    Arg:
        id1, id2: the two concepts 
    Returns:
        The similarity score        
    """
    in1 = set(getlinkedpages(id1, DIR_IN))
    out1 = set(getlinkedpages(id1, DIR_OUT))
    link1 = in1.union(out1)
    
    in2 = set(getlinkedpages(id2, DIR_IN))
    out2 = set(getlinkedpages(id2, DIR_OUT))
    link2 = in2.union(out2)
    
    f1 = len(link1)
    f2 = len(link2)
    if (f1==0) or (f2==0):
        return 0;
    
    f12=len(link1.intersection(link2))
    sim = (f12)/(f1+f2-f12);
    return sim


def getsim_emb(id1,id2, direction):
    """ Calculates the similarity between two concepts
    Arg:
        id1, id2: the two concepts
        direction: 0 for in, 1 for out, 2 for all
        
    Returns:
        The similarity score
    """
    em1 = concept_embedding(id1, direction);
    em2 = concept_embedding(id2, direction);
    if em1.empty or em2.empty:
        return 0;
    
    em1, em2 = em1.align(em2, fill_value=0)
#     print em1
#     print em2
    return 1-sp.spatial.distance.cosine(em1.values,em2.values);

def encode_entity(term, method,get_id=True):    
    if method in {'ngd'}:
        return term
    if get_id:
        term = title2id(term)
    if term is None:
        return None
    if method in {'rvspagerank','wlm','cocit','coup','ams'} :
        return term
    if 'word2vec_id' in method:
        term = 'id_' + str(term)
        return term
    if 'word2vec':
        term = str(term)
        return term
    
    return term

def getsim(id1,id2, method='rvspagerank', direction=DIR_BOTH, sim_method=None):
    """ Calculates well-known similarity metrics between two concepts 
    Arg:
        id1, id2: the two concepts 
        method:
            wlm: Wikipedia-Miner method
            cocit: cocitation
            coup: coupling
            ams: amsler
            rvspagerank: ebedding based similarity (in our case, 
                 reversed-page rank method)
    Returns:
        The similarity score        
    """
    log('[getsim started]\method = %s, direction = %s, id1=%s, id2=%s', method, direction, id1, id2)
    
    if method=='rvspagerank':
        sim = getsim_emb(id1,id2, direction)
    elif method=='wlm':
        sim = getsim_wlm(id1,id2)
    elif method=='cocit':
        sim = getsim_cocit(id1,id2)
    elif method=='coup':
        sim = getsim_coup(id1,id2)
    elif method=='ams':
        sim = getsim_ams(id1,id2)
    elif 'word2vec' in  method:
        sim = getsim_word2vec(id1, id2)
    elif 'ngd' in  method:
        sim = getsim_ngd(id1, id2)
    elif sim_method is not None:    
        sim = sim_method(id1,id2)
    else:
        sim=None
    log('[getsim]\tfinished')
    return sim

    
def getsim_file(infilename, outfilename, method='rvspagerank', direction=DIR_BOTH, sim_method=None):
    """ Batched (file) similarity.
    
    Args: 
        infilename: tsv file in the format of pair1    pair2   [goldstandard]
        outfilename: tsv file in the format of pair1    pair2   similarity
        direction: 0 for in, 1 for out, 2 for all
                        
    Returns:
        vector of scores, and Spearmans's correlation if goldstandard is given
    """
    log('[getsim_file started]\t%s -> %s', infilename, outfilename)
    outfile = open(outfilename, 'w');
    dsdata=readds(infilename);
    gs=[];
    scores=[];
    spcorr=None;
    for row in dsdata.itertuples():   
        log('processing %s, %s', row[1], row[2])
        if (row[1]=='null') or (row[2]=='null'):
            continue;
        if len(row)>3: 
            gs.append(row[3]);
            
        term1 = encode_entity(row[1], method, get_id=True)
        term2 = encode_entity(row[2], method, get_id=True)
            
        if (term1 is None) or (term2 is None):
            sim=0;
        else:
            sim=getsim(term1, term2, method, direction, sim_method);
        outfile.write("\t".join([str(row[1]), str(row[2]), str(sim)])+'\n')
        scores.append(sim)
    outfile.close();
    if gs:
        spcorr = sp.stats.spearmanr(scores, gs);
    log('[getsim_file]\tfinished')
    return scores, spcorr

    

def getembed_file(infilename, outfilename, direction, get_titles=False, cutoff=None):
    """ Batched (file) concept representation.
    
    Args: 
        infilename: tsv file in the format of pair1    pair2   [goldstandard]
        outfilename: tsv file in the format of pair1    pair2   similarity
        direction: 0 for in, 1 for out, 2 for all
        titles: include titles in the embedding (not needed for mere calculations)
        cutoff: the first top cutoff dimensions (None for all)        

    """
    
    log('[getembed_file started]\t%s -> %s', infilename, outfilename)
    outfile = open(outfilename, 'w');
    dsdata=readds(infilename, usecols=[0]);
    scores=[];
    for row in dsdata.itertuples():        
        wid = encode_entity(row[1], method='rvspagerank', get_id=True)
        if wid is None:
            em=pd.Series();
        else:
            em=conceptrep(wid, method='rvspagerank', direction = direction, 
                          get_titles = get_titles, cutoff=cutoff)
        outfile.write(row[1]+"\t"+em.to_json()+"\n")
    outfile.close();
    log('[getembed_file]\tfinished')



# A Simple Example

In [1]:
%load_ext autoreload
%autoreload 2
#%aimport calcsim

%aimport wikipedia

from wikipedia import * # uncomment
from calcsim import *   # uncomment
# Examples
reopen()
direction = DIR_OUT

page_title1 = 'Abortion' 
print ('page_title: ', page_title1)

page_id1 = title2id(page_title1)
print ("id: ", page_id1)

sr1 = synonymring_titles(page_id1)
print ("synonym ring: %s\n " % str(sr1[:5]))

rep1=conceptrep(page_id1, method='rvspagerank', direction = direction,  get_titles=True, cutoff=5)
print ("Concept Representation:  %s\n" % rep1.to_json())

print ("\n")

page_title2 = 'Miscarriage' 
print ('page_title: ', page_title2)

page_id2 = title2id(page_title2)
print ("id: ", page_id2)

sr2 = synonymring_titles(page_id2)
print ("synonym ring: %s\n " % str(sr2[:5]))

rep2=conceptrep(page_id2, method='rvspagerank', direction = direction,  get_titles=True, cutoff=5)
print ("Concept Representation: %s\n" % rep2.to_json())



sim = getsim(page_id1, page_id2,'rvspagerank',DIR_IN)
print ("similarity", sim)



('page_title: ', 'Abortion')
('id: ', 765L)
synonym ring: ('Abortion', 'Termination_of_pregnancy', 'Abortions', 'Induced_abortion', 'Abortionist')
 
Concept Representation:  {"765":["Abortion",0.0292042238],"30044455":["Therapeutic_abortion",0.0077943629],"492759":["Late_termination_of_pregnancy",0.0052510652],"4289811":["History_of_abortion",0.0052268208],"323793":["Abortifacient",0.0048862504]}



('page_title: ', 'Miscarriage')
('id: ', 144147L)
synonym ring: ('Miscarriage', 'Miscarreage', 'Threatened_abortion', 'Inevitable_abortion', 'Incomplete_abortion')
 
Concept Representation: {"144147":["Miscarriage",0.0389120779],"1771587":["Pregnancy",0.0136302541],"429542":["Preterm_birth",0.0123620717],"2300688":["Complications_of_pregnancy",0.0115965307],"54309":["Ectopic_pregnancy",0.0114757366]}

('similarity', 0.15728759037488615)


# Calculating All The Embeddngs
### Note: This step might take several days
This step can be safely skipped and let the caching mechansism happens gradually over time, but if you have some heavy task, it is worth to invest some time and calculate the embeddings off-line

### First. Get a list of the id pages
```
SELECT page_id
INTO OUTFILE '~/backup/wikipedia/20160305/edited/enwiki-20160305-page.dumped.ssv'
FROM page
where page_namespace=0 and page_is_redirect=0 ;
```
### Second. Starting the precalculation

In [ ]:
%%writefile preembed.py 
"""Pre calculation of the embeddings"""

from config import *

# %load_ext autoreload
# %autoreload

# %aimport calcsim
from calcsim import *
direction = DIR_IN;
dirstr = graphtype(direction)
#wid_fname  = os.path.join(home, 'backup/wikipedia/20160305/embed/enwiki-20160305-page.dumped.ssv')
wid_fname = os.path.join(home, 'backup/wikipedia/20160305/embed/enwiki-20160305-embeddings.'+dirstr+'.dead_2.ssv')

done_fname = os.path.join(home, 'backup/wikipedia/20160305/embed/enwiki-20160305-embeddings.'+dirstr+'.done.ssv')
dead_fname = os.path.join(home, 'backup/wikipedia/20160305/embed/enwiki-20160305-embeddings.'+dirstr+'.dead_3.ssv')
rewrite = True
lastwid = ""
if os.path.exists(done_fname):
    with open(done_fname) as done_f:
        for lastwid in done_f:
            pass
        if lastwid is not None:
            lastwid = lastwid.strip() 
            

wid_f = open(wid_fname)
done_f = open(done_fname, 'a')
dead_f = open(dead_fname, 'a')
    
if lastwid:
    for line in wid_f:
        if line.strip() == lastwid:
            break
    print "Continuing from ", lastwid
else: 
    print "Fresh start"
    
for line in wid_f:
    wid = line.strip().split('\t')[0]
    if rewrite:
        deletefromcache(wid, direction)
    em = concept_embedding(wid, direction)
    if em.empty:
        count = str(len(getlinkedpages(wid, direction)))
        dead_f.write(wid+'\t'+id2title(wid)+'\t'+count+'\n')
    done_f.write(wid+'\n')
wid_f.close()
done_f.close()
dead_f.close()

print "done"

# Visualizing The Embeddings

In [ ]:
%load_ext autoreload
%autoreload

from calcsim import *

import json
from IPython.display import Javascript

cre1 = conceptrep(title2id('Tehran'), method='rvspagerank', direction = DIR_OUT, get_titles=True, cutoff=5);
cre2 = conceptrep(title2id('Sanandaj'), method='rvspagerank', direction = DIR_OUT, get_titles=True, cutoff=5);


#runs arbitrary javascript, client-side
Javascript("""
           window.vizObj1={};window.vizObj2={};
           """.format(cre1.to_json(), cre2.to_json()))


In [ ]:
%%javascript

require.config({
    paths: {
        d3:'//cgm6.research.cs.dal.ca/~sajadi/wikisim/js/d3',
        d3_cloud:'//cgm6.research.cs.dal.ca/~sajadi/wikisim/js/d3.layout.cloud',
        simple_draw:'//cgm6.research.cs.dal.ca/~sajadi/wikisim/js/simpledraw'

    }
});

In [ ]:
%%javascript

function createWords(cp){

    var titles=[];
    var scores=[];

    for (var key in cp){ 
        if (cp.hasOwnProperty(key)) {
            titles.push(cp[key][0])
            scores.push(cp[key][1])
        }
    }
    var sum = scores.reduce(function(a, b) {return a + b;});
    var min = Math.min.apply(null, scores)
    var max = Math.max.apply(null, scores)
    
    scores=scores.map(function(a){return (a/sum)*90+20});
    var words=[];
    for (var i = 0; i<titles.length; i++) {
        words.push({"text":titles[i], "size": scores[i]})
    }
    return words;
}

var words1=createWords(window.vizObj1);
//element.text(JSON.stringify(words1));
var words2=createWords(window.vizObj2);
require(['d3','d3_cloud', 'simple_draw'], function(d3,d3_cloud, simple_draw){
    $("#chart1").remove();
    element.append("<div id='chart1' style='width:49%; height:500px; float:left; border-style:solid'> </div>");
    simpledraw(words1, chart1);
    
    $("#chart2").remove();
    element.append("<div id='chart2' style='width:49%; margin-left:2%; height:500px; float:left; border-style:solid'> </div>");
    simpledraw(words2, '#chart2');    
    
});    
    
